# QnA Model

질문 생성 https://huggingface.co/Sehong/kobart-QuestionGeneration

응답 https://huggingface.co/bespin-global/klue-bert-base-aihub-mrc

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import pipeline
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# 질문 생성
tokenizer_Q = PreTrainedTokenizerFast.from_pretrained('Sehong/kobart-QuestionGeneration')
model_Q = BartForConditionalGeneration.from_pretrained('Sehong/kobart-QuestionGeneration')

# paragraph = "1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. <unused0> 1989년 2월 15일"
paragraph = "애플 M2(Apple M2)는 애플이 설계한 중앙 처리 장치(CPU)와 그래픽 처리 장치(GPU)의 ARM 기반 시스템이다.인텔 코어(Intel Core)에서 맥킨토시 컴퓨터용으로 설계된 2세대 ARM 아키텍처이다. 애플은 2022년 6월 6일 WWDC에서 맥북 에어, 13인치 맥북 프로와 함께 M2를 발표했다. 애플 M1의 후속작이다. M2는 TSMC의 '향상된 5나노미터 기술' N5P 공정으로 만들어졌으며, 이전 세대 M1보다 25% 증가한 200억개의 트랜지스터를 포함하고 있으며, 최대 24기가바이트의 RAM과 2테라바이트의 저장공간으로 구성할 수 있다. 8개의 CPU 코어(성능 4개, 효율성 4개)와 최대 10개의 GPU 코어를 가지고 있다. M2는 또한 메모리 대역폭을 100 GB/s로 증가시킨다. 애플은 기존 M1 대비 CPU가 최대 18%, GPU가 최대 35% 향상됐다고 주장하고 있으며,[1] 블룸버그통신은 M2맥스에 CPU 코어 12개와 GPU 코어 38개가 포함될 것이라고 보도했다."
# paragraph = "전화 주셔서 감사합니다. 혹시 이거 탈퇴는 어떻게 하나요. 애 아빠랑 둘 다 가입을 해버려서 한쪽은 탈퇴해야 해서요. 홈페이지 접속 중이신가요 고객님. 네 로그인 해둔 상태예요. 마이페이지 가셔서 회원정보 수정 들어가주시겠어요. 아 여기 아래에 있네요. 네 거기서 탈퇴 진행하시면 됩니다. 확인했습니다 끊을게요 수고하세요."

raw_input_ids = tokenizer_Q.encode(paragraph)
input_ids = [tokenizer_Q.bos_token_id] + raw_input_ids + [tokenizer_Q.eos_token_id]

summary_ids = model_Q.generate(torch.tensor([input_ids]))
print('Question: ', tokenizer_Q.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True),'?')


# 응답
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def predict_answer(qa_text_pair):
    # Encoding
    encodings = tokenizer(paragraph, tokenizer_Q.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True), 
                      max_length=512, 
                      truncation=True,
                      padding="max_length", 
                      return_token_type_ids=False,
                      return_offsets_mapping=True
                      )
    encodings = {key: torch.tensor([val]).to(device) for key, val in encodings.items()}             

    # Predict
    pred = model(encodings["input_ids"], attention_mask=encodings["attention_mask"])
    start_logits, end_logits = pred.start_logits, pred.end_logits
    token_start_index, token_end_index = start_logits.argmax(dim=-1), end_logits.argmax(dim=-1)
    pred_ids = encodings["input_ids"][0][token_start_index: token_end_index + 1]
    answer_text = tokenizer.decode(pred_ids)

    # Offset
    answer_start_offset = int(encodings['offset_mapping'][0][token_start_index][0][0])
    answer_end_offset = int(encodings['offset_mapping'][0][token_end_index][0][1])
    answer_offset = (answer_start_offset, answer_end_offset)
 
    return {'answer_text':answer_text, 'answer_offset':answer_offset}


## Load fine-tuned MRC model by HuggingFace Model Hub ##
HUGGINGFACE_MODEL_PATH = "bespin-global/klue-bert-base-aihub-mrc"
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model = AutoModelForQuestionAnswering.from_pretrained(HUGGINGFACE_MODEL_PATH).to(device)

qa_text_pair = {'paragraph':paragraph, 'question':tokenizer_Q.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)}
result = predict_answer(qa_text_pair)
print('Answer: ', result['answer_text'])  # 기존 M1 대비 CPU가 최대 18 %, GPU가 최대 35 % 향상

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Question:  M2의 후속작인 M2는 기존 M1보다 무엇이 최대 18% 향상 ?
Answer:  CPU
